In [1]:
import warnings
import sys
import requests
from io import StringIO, BytesIO
import re
from six import u
from bs4 import BeautifulSoup
import json
import pandas as pd
import gc


from datetime import datetime
warnings.filterwarnings("ignore")
today = datetime.utcnow().isoformat()


In [2]:
requestSession = requests.Session()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Referer': 'https://cookpad.com/eg/%D9%88%D8%B5%D9%81%D8%A7%D8%AA',
    'Turbolinks-Referrer': 'https://cookpad.com/eg/%D9%88%D8%B5%D9%81%D8%A7%D8%AA',
    'X-Turbolinks-Request': 'true',
    'Accept': 'text/html, application/xhtml+xml',
    'Accept-Encoding': 'gzip, deflate, sdch, br',
    'Accept-Language': 'en-US,en;q=0.8,ar;q=0.6'
    }
def spider(url):
   
    r = requestSession.get(url=url, headers=headers, verify=False, timeout=(3.05, 27))
    r.raise_for_status()
    
    page = r.text
-
def saveToFile(data,fileName):
    with open(fileName, 'w', encoding="utf-8") as fo:
        json.dump(data, fo, ensure_ascii=False)
        fo.flush()

    

In [3]:
counter = 1
data=[]

with open("links.txt") as f:
    for line in f:
        try:
            #print('{}:{}'.format(counter, line))
            data.append(spider(line))
            counter = counter + 1
            print(counter)
            if(counter % 1000 == 0):
                print(counter)
                saveToFile(data, 'output/test{}{}.json'.format(counter, datetime.now().strftime("%Y%m%d-%H%M%S")))
                del data [:]
                print(gc.collect())
        except:
            print ("Unexpected error:", sys.exc_info())
            counter = counter - 1
            continue
    print(counter)
    saveToFile(data, 'output/test{}{}.json'.format(counter, datetime.now().strftime("%Y%m%d-%H%M%S")))
    del data [:]
    print(gc.collect())            
        
          


2
2
2802
